In [ ]:
from collections import deque, defaultdict
from sortedcontainers import SortedList

class Router:

    def __init__(self, memoryLimit: int):
        self.memoryLimit = memoryLimit
        self.queue = deque()
        self.packetSet = set()
        self.destTimeMap = defaultdict(SortedList) 
        

    def addPacket(self, source: int, destination: int, timestamp: int) -> bool:
        key = (source, destination, timestamp)
        if key in self.packetSet:
            return False
        if len(self.queue) >= self.memoryLimit:
            old_src, old_dst, old_ts = self.queue.popleft()
            self.packetSet.remove((old_src,old_dst,old_ts))
            self.destTimeMap[old_dst].remove(old_ts)
            if not self.destTimeMap[old_dst]:
                del self.destTimeMap[old_dst]  # clean up

        self.queue.append((source, destination, timestamp))
        self.packetSet.add(key)
        self.destTimeMap[destination].add(timestamp)
        return True


    def forwardPacket(self) -> List[int]:
        if not self.queue:
            return []
        src,dst,ts= self.queue.popleft()
        self.packetSet.remove((src,dst,ts))
        self.destTimeMap[dst].remove(ts)
        if not self.destTimeMap[dst]:
            del self.destTimeMap[dst]
        return [src, dst, ts]


        

    def getCount(self, destination: int, startTime: int, endTime: int) -> int:
        times = self.destTimeMap[destination]
        left = times.bisect_left(startTime)
        right = times.bisect_right(endTime)
        return right - left
